In [1]:
%%capture
import sys
sys.path.append('/work/pikachu/utils/')
sys.path.append('/work/pikachu/third/')
sys.path.append('..')
from gezi.common import *
import gezi
from src import config
from src.config import *
from src import util
from src.dataset import Dataset
from src.eval import eval_seq, calc_score
from src.tf.decode import adjust_pad
gezi.init_flags()

In [3]:
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
  tf.config.set_visible_devices([physical_devices[x] for x in range(4)], 'GPU')
tf.config.set_soft_device_placement(True)
for gpu in physical_devices:
  tf.config.experimental.set_memory_growth(gpu, True)

In [26]:
# model = 'conformer-fat.encoder_layers-15.time_reduce.time_reduce_idx-7.relpos_combine_mode-2.fixreduce.eval'
# model = 'conformer-subsample.encoder_layers-15.len_loss_weight-1.eval'
model = 'final-17layers.finetune.eval'
model_dir = f'../working/offline/30/0/{model}'

In [28]:
!ls {model_dir}

/bin/bash: /home/gezi/miniconda3/envs/tf/lib/libtinfo.so.6: no version information available (required by /bin/bash)
args.txt	       eval.csv       flags.txt		 log.txt      src
ckpt		       eval_step.txt  global.json	 metrics.csv
command.txt	       flags.json     history_flags.txt  outputs.npy
dataset-metadata.json  flags.pkl      log.html		 script.txt


In [29]:
outputs = gezi.load(f'{model_dir}/outputs.npy')
outputs.shape

(15045, 80, 61)

In [37]:
df = pd.read_csv(f'{model_dir}/eval.csv')
df

,sequence_id,phrase_type,phrase_dup,phrase_true,phrase_pred,phrase_ori,char/max_idx,char/ori_rate,char/true_rate,char/pred_rate,...,phrase_len_rate,idx,distance,acc/char,acc/type,acc/first,acc/last,n_frame,frame_mean,score
0,1840954014,phone,0,909-205-3229,909-205-3229,<PAD><PAD><PAD><PAD><PAD>9<PAD><PAD>0<PAD>9<PA...,33,0.4125,2.750000,2.750000,...,1.000000,173,0,1.000000,True,True,True,86,0.209623,1.000000
1,1828607689,phone,0,889-431-5361,889-471-5361,<PAD><PAD><PAD><PAD><PAD>8<PAD><PAD>8<PAD><PAD...,39,0.4875,3.250000,3.250000,...,1.000000,81,1,0.983051,True,True,True,124,0.206578,0.916667
2,1847541487,address,1,6558 cinglie,cinglie,<PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><...,26,0.3250,2.166667,3.714286,...,0.583333,211,5,0.932203,True,False,True,51,0.266139,0.583333
3,1831041359,url,0,http://alquilerbarbate.es,https://alpralebawte.es,ht<PAD>ttp<PAD>s:/<PAD><PAD><PAD>/<PAD><PAD><P...,57,0.7125,2.280000,2.478261,...,0.920000,108,8,0.932203,True,True,True,191,0.151220,0.680000
4,1817672186,phone,0,596-033-4046,+56-033-40447,+<PAD><PAD><PAD><PAD><PAD>55<PAD><PAD><PAD><PA...,53,0.6625,4.416667,4.076923,...,1.083333,4,4,0.949153,True,False,False,181,0.229389,0.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15040,385867160,address,1,4051 concord street north,465 m can cos,<PAD><PAD><PAD><PAD><PAD><PAD>4<PAD>6<PAD><PAD...,36,0.4500,1.440000,2.769231,...,0.520000,15032,19,0.830508,True,True,False,77,0.216230,0.240000
15041,385641045,url,1,www.skipcity-dcf.jp,www.skypcate-dc.jp,w<PAD>w<PAD><PAD>w<PAD><PAD><PAD><PAD><PAD><PA...,66,0.8250,3.473684,3.666667,...,0.947368,15030,4,0.932203,True,True,True,243,0.151183,0.789474
15042,382156022,address,0,9686 trigg lake,alan-sate,<PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><...,53,0.6625,3.533333,5.888889,...,0.600000,14999,13,0.813559,True,False,True,180,0.152415,0.133333
15043,359820834,address,1,42 sanderling,42 sander ling,<PAD><PAD><PAD><PAD>4<PAD>2 ss<PAD>a<PAD>n<PA...,33,0.4125,2.538462,2.357143,...,1.076923,14828,1,1.000000,True,True,True,66,0.208407,0.923077


In [38]:
df.sample(frac=1).head(10)

,sequence_id,phrase_type,phrase_dup,phrase_true,phrase_pred,phrase_ori,char/max_idx,char/ori_rate,char/true_rate,char/pred_rate,...,phrase_len_rate,idx,distance,acc/char,acc/type,acc/first,acc/last,n_frame,frame_mean,score
6639,1758826863,address,1,754 e county 17 3/4 st,754 ne county r wst,<PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD>77<PAD...,70,0.8750,3.181818,3.500000,...,0.909091,6620,7,0.915254,True,True,True,271,0.163065,0.681818
4667,968922764,url,0,www.dibujotecnico.mx,www.dibgantecned.mx/,w<PAD>ww<PAD>w<PAD><PAD><PAD><PAD><PAD>.<PAD><...,53,0.6625,2.650000,2.650000,...,1.000000,4600,7,0.898305,True,True,False,177,0.180733,0.650000
9762,2021084945,address,1,601 goode hollow,601 goode hollow,<PAD><PAD><PAD><PAD><PAD><PAD><PAD>6<PAD><PAD>...,47,0.5875,2.937500,2.937500,...,1.000000,9755,0,1.000000,True,True,True,166,0.219213,1.000000
2868,679389145,url,1,ngotenders.net/mayatestorreon,ngotenders.net/mayatestorreron,<PAD><PAD><PAD><PAD><PAD>nn<PAD>gg<PAD>o<PAD><...,79,0.9875,2.724138,2.633333,...,1.034483,2749,1,1.000000,True,True,True,332,0.290887,0.965517
13282,993840101,url,1,kytucxa.saodo.edu.vn/gymade,tucgaeadu.edu.vn/pyman,<PAD><PAD><PAD><PAD><PAD><PAD>t<PAD>uu<PAD><PA...,74,0.9250,2.740741,3.363636,...,0.814815,13261,10,0.915254,True,False,False,371,0.217150,0.629630
1331,1306337800,address,1,522 maurepas drive,52 maurepas drive,<PAD><PAD><PAD><PAD><PAD><PAD>55<PAD>2<PAD><PA...,47,0.5875,2.611111,2.764706,...,0.944444,1125,1,1.000000,True,True,True,138,0.189898,0.944444
1762,2065567989,address,0,47 swinnea lake drive,eti iin,<PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><...,32,0.4000,1.523810,4.571429,...,0.333333,1725,18,0.813559,True,False,False,6,0.229128,0.142857
6140,174045795,url,0,http://lnx.lazialita.com,http://lnca.laz.ialita.com,ht<PAD>tpp<PAD>::/<PAD><PAD>//<PAD><PAD><PAD><...,58,0.7250,2.416667,2.230769,...,1.083333,6197,3,0.983051,True,True,True,182,0.228133,0.875000
13358,1017579885,url,1,www.hongwulian.com/profi/15694,www.hanguliiat.com/profi/15694,w<PAD><PAD>ww<PAD>ww<PAD><PAD><PAD><PAD><PAD><...,78,0.9750,2.600000,2.600000,...,1.000000,13439,4,0.983051,True,True,True,347,0.200318,0.866667
11312,1354565787,address,1,116 north la tomas court,16 north la topis court,<PAD><PAD><PAD><PAD>111<PAD>6<PAD> n<PAD>or<P...,52,0.6500,2.166667,2.260870,...,0.958333,11373,3,0.949153,True,True,True,159,0.170255,0.875000


In [39]:
df.columns

Index(['sequence_id', 'phrase_type', 'phrase_dup', 'phrase_true',
       'phrase_pred', 'phrase_ori', 'char/max_idx', 'char/ori_rate',
       'char/true_rate', 'char/pred_rate', 'phrase_len_true',
       'phrase_len_pred', 'phrase_len_pred_', 'phrase_len_pred2_',
       'phrase_len_rate', 'idx', 'distance', 'acc/char', 'acc/type',
       'acc/first', 'acc/last', 'n_frame', 'frame_mean', 'score'],
      dtype='object')

In [40]:
df['logits'] = list(outputs)

In [41]:
cols = ['sequence_id', 'phrase_type', 'phrase_dup', 'phrase_true', 'phrase_pred', 'score', 'distance', 'phrase_len_true', 'phrase_len_pred', 'phrase_ori', 'logits']
df = df[cols]

In [42]:
df[df.sequence_id==1788518982]

,sequence_id,phrase_type,phrase_dup,phrase_true,phrase_pred,score,distance,phrase_len_true,phrase_len_pred,phrase_ori,logits
294,1788518982,phone,0,304,584 s,0.0,4,3,5,<PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><...,"[[6.1061535, -6.434418, -20.475275, -22.907469..."


In [43]:
df[df.score < 0.6].sample(frac=1).head(10)

,sequence_id,phrase_type,phrase_dup,phrase_true,phrase_pred,score,distance,phrase_len_true,phrase_len_pred,phrase_ori,logits
3753,1899704414,url,1,www.acupunctuur-baf.be,www.acpertchul-baf.cm,0.590909,9,22,21,w<PAD><PAD>w<PAD><PAD>w<PAD><PAD><PAD><PAD>.<P...,"[[1.5519195, -1.1837002, -12.793558, -9.310667..."
11606,484689345,phone,0,+297-31-25-42,4 sost 1an,0.000000,13,13,10,<PAD><PAD><PAD><PAD>4<PAD><PAD><PAD><PAD> <PAD...,"[[4.688967, -5.7557693, -17.123913, -18.945042..."
6791,2002464766,phone,0,+237-7620-612-48,+610-108-40,0.375000,10,16,11,+<PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD>...,"[[3.8790522, -3.1370993, -11.488737, -14.84300..."
12833,1589095484,phone,0,546-492-6071,rilas,0.000000,12,12,5,<PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><...,"[[3.7842073, -5.44141, -16.826958, -16.667307,..."
6840,1990814188,address,0,864 smith booth street,864 smyth cook,0.500000,11,22,14,<PAD><PAD><PAD><PAD><PAD>8<PAD>6<PAD>4<PAD> s...,"[[7.5174317, -7.382164, -26.014736, -28.605946..."
13218,616467622,address,1,7398 dinebito,28 zi abil,0.307692,9,13,11,<PAD><PAD><PAD><PAD><PAD><PAD><PAD>22<PAD><PAD...,"[[7.934916, -8.940137, -28.83297, -32.757698, ..."
12549,1166625603,address,1,55 kaaf,5 f,0.428571,4,7,4,<PAD><PAD><PAD><PAD><PAD><PAD><PAD>5<PAD><PAD>...,"[[6.3188143, -8.011264, -24.216774, -23.602005..."
14523,97227934,phone,0,+44-80-81-1546-502,4 so 1arn,0.111111,16,18,9,<PAD><PAD><PAD><PAD>4<PAD><PAD><PAD><PAD> s<PA...,"[[5.3350663, -4.5142107, -16.46523, -17.315268..."
7119,1733660616,address,1,582 sumner ms 321,582 spulber pc 34,0.588235,7,17,17,<PAD><PAD><PAD><PAD><PAD>5<PAD><PAD>8<PAD>2<PA...,"[[12.183031, -10.712164, -35.108803, -38.42448..."
1696,2061640657,address,1,melinda_dellert,44-016-3041,0.000000,15,15,11,<PAD><PAD>4<PAD><PAD>4<PAD><PAD><PAD>-0<PAD>1<...,"[[5.9212923, -5.706414, -15.35202, -18.188091,..."


In [44]:
df = df.sort_values(by=['score'], ascending=True)
df.head(10)

,sequence_id,phrase_type,phrase_dup,phrase_true,phrase_pred,score,distance,phrase_len_true,phrase_len_pred,phrase_ori,logits
7742,561191587,address,0,5437 atenas,+55-555-5555,0.0,11,11,12,+<PAD>5<PAD>5<PAD><PAD><PAD><PAD>-5<PAD>5<PAD>...,"[[3.5598445, -4.890339, -12.272217, -14.447025..."
11056,1483137807,phone,0,+998-53-1956-5551,e a ae oeon,0.0,17,17,11,<PAD><PAD>e<PAD><PAD><PAD><PAD><PAD><PAD> <PAD...,"[[3.2871158, -4.547524, -17.786144, -18.84295,..."
11040,391515081,phone,0,153-808-1119,e si aoei,0.0,12,12,9,<PAD><PAD>e<PAD><PAD><PAD><PAD><PAD><PAD> <PAD...,"[[3.3294668, -4.3978286, -17.916544, -19.56990..."
6174,171946077,phone,0,+880-562-7687,we con,0.0,13,13,6,<PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><...,"[[4.697115, -5.3360686, -16.660582, -17.192085..."
10975,418308530,phone,0,123-715-4563,we e o o een,0.0,12,12,12,w<PAD>e<PAD><PAD><PAD><PAD><PAD><PAD> <PAD><PA...,"[[2.992219, -4.3854513, -16.50362, -18.048286,..."
10952,392807612,phone,0,+48-859-455-700,saada,0.0,15,15,5,<PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><...,"[[3.9431744, -5.951052, -17.168646, -17.051357..."
2218,920740683,phone,0,331-553-1509,6 se cad,0.0,12,12,8,6<PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD> s<PA...,"[[4.027251, -5.538156, -16.980179, -17.793324,..."
2231,903056842,address,1,216378 hains,e maeis itee,0.0,12,12,12,<PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><...,"[[4.577347, -4.4581556, -20.262148, -21.338902..."
3200,645463392,phone,0,685-304-1532,wtt.:/wo.com,0.0,12,12,12,w<PAD>t<PAD>t<PAD><PAD><PAD><PAD>.<PAD><PAD>:/...,"[[2.5845375, -3.579776, -11.799004, -11.709607..."
725,1547340023,phone,0,251-777-8515,so cod,0.0,12,12,7,<PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD> ...,"[[4.9681544, -5.6842065, -17.306747, -16.52384..."


In [45]:
df.tail(10)

,sequence_id,phrase_type,phrase_dup,phrase_true,phrase_pred,score,distance,phrase_len_true,phrase_len_pred,phrase_ori,logits
9383,1938830135,phone,0,160-713-2395,160-713-2395,1.0,0,12,12,<PAD><PAD><PAD><PAD><PAD>1<PAD>6<PAD>0<PAD><PA...,"[[14.698955, -10.1479645, -36.48256, -41.78263..."
3000,63947308,url,1,m.iceporn.com/lashermes,m.iceporn.com/lashermes,1.0,0,23,23,<PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD>mm<PAD...,"[[15.785314, -10.161469, -33.701065, -31.93944..."
9378,1923210982,address,0,jonah nielsen,jonah nielsen,1.0,0,13,13,<PAD><PAD><PAD><PAD><PAD>j<PAD>oonn<PAD>a<PAD>...,"[[16.500963, -7.77636, -42.41331, -44.332253, ..."
3003,41538558,url,1,valentina-nappi-jeans.php,valentina-nappi-jeans.php,1.0,0,25,25,<PAD><PAD><PAD><PAD><PAD>v<PAD>aallleen<PAD>t<...,"[[16.512646, -8.418536, -37.204926, -36.57923,..."
9375,1936959664,address,1,911050 hounddog lane,911050 hounddog lane,1.0,0,20,20,<PAD><PAD><PAD>9<PAD>1<PAD><PAD>110<PAD>5500 <...,"[[11.077019, -8.051785, -37.667324, -37.68607,..."
9373,1937306384,address,0,calvin mora,calvin mora,1.0,0,11,11,<PAD><PAD><PAD><PAD><PAD>ccall<PAD>v<PAD>i<PAD...,"[[17.448391, -7.950071, -38.11001, -37.75035, ..."
9372,1917439774,address,0,johnny rivers,johnny rivers,1.0,0,13,13,<PAD><PAD><PAD><PAD><PAD>j<PAD>ohhn<PAD><PAD>n...,"[[14.626737, -6.1368246, -37.604847, -38.60217..."
9370,1939984762,address,0,619 erdine,619 erdine,1.0,0,10,10,<PAD><PAD><PAD><PAD><PAD><PAD><PAD>66<PAD><PAD...,"[[15.363209, -10.051987, -35.969887, -33.97941..."
2942,2142560267,address,0,vince house,vince house,1.0,0,11,11,<PAD><PAD><PAD><PAD><PAD><PAD>v<PAD>inncccee<P...,"[[13.826261, -6.5978217, -37.53155, -35.386013..."
0,1840954014,phone,0,909-205-3229,909-205-3229,1.0,0,12,12,<PAD><PAD><PAD><PAD><PAD>9<PAD><PAD>0<PAD>9<PA...,"[[14.899874, -9.280058, -32.50017, -39.11643, ..."


In [46]:
def eval_score(phrase_true, phrase_pred):
  distance = calc_score(phrase_true, phrase_pred)
  ic(distance, len(phrase_true), len(phrase_pred), phrase_true, phrase_pred)
  score = max(len(phrase_true) - distance, 0.) / len(phrase_true)
  return score

In [47]:
def show(probs):
  for i, prob in enumerate(probs):
    vals, idxes = gezi.topk(prob, 7)
    print(i, end=' | ')
    for val, idx in zip(vals, idxes):
      val = '%.2f' % val
      print(IDX2CHAR[idx], val, sep=' ', end=' | ')
    print()
    
def debug(i):
  logits = df.iloc[i].logits
  ic(df.iloc[i])
  probs = gezi.softmax(logits)
  show(probs)

In [48]:
def adjust_pad(pred, pad_rate):
  pred = tf.nn.softmax(pred)
  pred0, pred1 = pred[..., 0:1], pred[..., 1:]
  pred0 *= pad_rate
  pred = tf.concat([pred0, pred1], axis=-1)
  if FLAGS.ctc_decode_method.startswith('tf-'):
    pred = tf.math.log(pred + 1e-6) + math.log(10.)
  return pred

def decode(x, pad_rate=1):
  if pad_rate != 1:
    x = adjust_pad(x, pad_rate)
  x = tf.argmax(x, axis=-1)
  x = tf.concat([x, tf.zeros((1), dtype=x.dtype)], axis=0)
  diff = tf.not_equal(x[:-1], x[1:])
  diff = tf.not_equal(x[:-1], x[1:])
  adjacent_indices = tf.where(diff)[:, 0]
  x = tf.gather(x, adjacent_indices)
  mask = x != PAD_IDX
  x = tf.boolean_mask(x, mask, axis=0)
  x = x.numpy()
  x = [IDX2CHAR[a] for a in x]
  return x

In [55]:
debug(2000)

[08/23/23 23:32:29] 693575459.py:12 in debug()
                    df.iloc[i]: sequence_id                                                203694832
                                phrase_type                                                  address
                                phrase_dup                                                         0
                                phrase_true                                       1256 s jeftoria rd
                                phrase_pred                                       6 ast fitoria road
                                score                                                       0.444444
                                distance                                                          10
                                phrase_len_true                                                   18
                                phrase_len_pred                                                   18
                                phrase_ori  

0 | <PAD> 0.62 | 6 0.13 | 1 0.06 | 2 0.04 | 4 0.03 | 5 0.03 | 7 0.03 | 
1 | <PAD> 0.93 | 6 0.01 | 1 0.01 | 8 0.01 | 2 0.01 | 5 0.01 | 0 0.00 | 
2 | <PAD> 0.43 | 6 0.21 | 2 0.07 | 1 0.06 | 5 0.06 | 4 0.05 | 8 0.03 | 
3 | 6 0.51 | <PAD> 0.37 | 2 0.04 | 7 0.02 | 4 0.01 | 8 0.01 | 5 0.01 | 
4 | <PAD> 0.56 | 6 0.39 | 2 0.02 | 7 0.01 | 4 0.01 | 8 0.00 | 0 0.00 | 
5 | <PAD> 0.77 | 6 0.16 | 2 0.02 | 7 0.02 | 4 0.01 | 8 0.00 | 1 0.00 | 
6 | <PAD> 0.42 | 6 0.34 | 2 0.09 | 7 0.06 | 4 0.03 | 8 0.02 | 9 0.01 | 
7 | <PAD> 0.75 | 6 0.17 | 2 0.03 | 8 0.01 | 7 0.01 | 4 0.01 | 0 0.01 | 
8 |   0.89 | <PAD> 0.05 | 6 0.03 | 0 0.01 | 2 0.01 | 4 0.00 | 1 0.00 | 
9 |   0.87 | w 0.09 | <PAD> 0.02 | e 0.01 | n 0.00 | m 0.00 | s 0.00 | 
10 | <PAD> 0.56 | w 0.15 | n 0.10 | e 0.07 | m 0.03 | s 0.03 | c 0.02 | 
11 | a 0.81 | <PAD> 0.09 | e 0.07 | o 0.02 | s 0.00 | n 0.00 | i 0.00 | 
12 | <PAD> 0.70 | a 0.23 | s 0.04 | l 0.02 | r 0.00 | n 0.00 | t 0.00 | 
13 | s 0.55 | <PAD> 0.21 | l 0.14 | t 0.06 | r 0.02 | n 0.01 

In [ ]:
df.iloc[0].logits.shape

In [ ]:
eval_score(df.iloc[1].phrase_true, ''.join(decode(df.iloc[1].logits, 0.2)))

In [ ]:
eval_score(df.iloc[1].phrase_true, ''.join(decode(df.iloc[1].logits, 0.1)))

In [ ]:
eval_score('242-197-6202', '2217 whon')

In [ ]:
import numpy as np
import time

chars = list(IDX2CHAR.values())
rev_mapping = IDX2CHAR.copy()
mapping = CHAR2IDX.copy()
rev_mapping[0] = '我'
rev_mapping[60] = '你'
mapping = {v: k for k, v in rev_mapping.items()}
def beam_search(mat, beam_width=5):
    """Beam search decoder.

    See the paper of Hwang et al. and the paper of Graves et al.

    Args:
        mat: Output of neural network of shape TxC.
        chars: The set of characters the neural network can recognize, excluding the CTC-blank.
        beam_width: Number of beams kept per iteration.
        lm: Character level language model if specified.

    Returns:
        The decoded text.
    """

    blank_idx = 0
    max_T, max_C = mat.shape
    # blank_idx = max_C - 1
    # last_chars_map = np.arange(max_C - 1)
    last_chars_map = np.arange(1, max_C)
    # initialise beam state
    labels = ['']
    pr_blanks = np.array([1])
    pr_totals = np.array([1])
    pr_non_blanks = np.array([0])

    new_pr_blanks = np.zeros((beam_width * (len(chars))))
    new_pr_totals = np.zeros((beam_width * (len(chars))))
    new_pr_non_blanks = np.zeros((beam_width * (len(chars))))

    # go over all time-steps
    for t in range(0, max_T):
        # get beam-labelings of best beams
        if len(labels) <= beam_width:
            ixs = np.arange(len(labels))
        else:
            ixs = np.argpartition(pr_totals, -beam_width)[-beam_width:]

        new_labels = []

        # go over best beams
        stored_ixs = []
        for j, ix in enumerate(ixs):
            # probability of paths ending with a non-blank
            pr_non_blank = 0
            # in case of non-empty beam
            if len(labels[ix]) > 0:
                # probability of paths with repeated last char at the end
                # pr_non_blank = pr_non_blanks[ix] * mat[t, mapping[labels[ix][-1]]]
                pr_non_blank = pr_non_blanks[ix] * mat[t, mapping[labels[ix][-1]]]

            # probability of paths ending with a blank
            pr_blank = pr_totals[ix] * mat[t, blank_idx]

            # fill in data for current beam
            new_labels.append(labels[ix])
            new_pr_non_blanks[j * max_C] = pr_non_blank
            new_pr_blanks[j * max_C] = pr_blank
            new_pr_totals[j * max_C] = pr_blank + pr_non_blank

            new_labeling = [labels[ix] + rev_mapping[c] for c in range(1, max_C)]

            pr_non_blank = pr_totals[ix] * mat[t, last_chars_map]
            if len(labels[ix]) > 0:
                pr_non_blank[mapping[labels[ix][-1]] - 1] = pr_blanks[ix] * mat[t, mapping[labels[ix][-1]] - 1]

            new_labels += new_labeling
            new_pr_non_blanks[j * max_C + 1:(j + 1) * max_C] = pr_non_blank
            new_pr_totals[j * max_C + 1:(j + 1) * max_C] = pr_non_blank
            new_pr_blanks[j * max_C + 1:(j + 1) * max_C] = 0
            
        ic(t, new_labels)

        unique_labels, inverse = np.unique(new_labels, return_inverse=True, axis=0)
        label_count = unique_labels.shape[0]

        pr_blanks = np.zeros(label_count)
        pr_totals = np.zeros(label_count)
        pr_non_blanks = np.zeros(label_count)

        np.add.at(pr_blanks, inverse, new_pr_blanks[:len(new_labels)])
        np.add.at(pr_totals, inverse, new_pr_totals[:len(new_labels)])
        np.add.at(pr_non_blanks, inverse, new_pr_non_blanks[:len(new_labels)])

        labels = np.array([label for label in unique_labels])

    # sort by probability
    ic(labels)
    best_labeling = labels[np.argmax(pr_totals)]

    # map label string to char string
    # return np.array([IDX2CHAR[c] for c in best_labeling], dtype=np.int64)
    return best_labeling


In [ ]:
import numpy as np
import time

chars = list(IDX2CHAR.values())
chars = [x for x in chars if not x.startswith('<')]
chars += ['<EOS>']
# ic(chars)
rev_mapping = {k - 1: v for k, v in IDX2CHAR.items()}
# rev_mapping[0] = '我'
# rev_mapping[60] = '你'
mapping = {v: k for k, v in rev_mapping.items()}

def beam_search(mat, beam_width=1):
    """Beam search decoder.

    See the paper of Hwang et al. and the paper of Graves et al.

    Args:
        mat: Output of neural network of shape TxC.
        chars: The set of characters the neural network can recognize, excluding the CTC-blank.
        beam_width: Number of beams kept per iteration.
        lm: Character level language model if specified.

    Returns:
        The decoded text.
    """
    blank_idx = len(chars)
    max_T, max_C = mat.shape
    blank_idx = max_C - 1
    last_chars_map = np.arange(max_C - 1)

    # initialise beam state
    labels = ['']
    pr_blanks = np.array([1])
    pr_totals = np.array([1])
    pr_non_blanks = np.array([0])

    new_pr_blanks = np.zeros((beam_width * (len(chars) + 1)))
    new_pr_totals = np.zeros((beam_width * (len(chars) + 1)))
    new_pr_non_blanks = np.zeros((beam_width * (len(chars) + 1)))

    # go over all time-steps
    for t in range(0, max_T):
        # get beam-labelings of best beams
        if len(labels) <= beam_width:
            ixs = np.arange(len(labels))
        else:
            ixs = np.argpartition(pr_totals, -beam_width)[-beam_width:]

        new_labels = []

        # go over best beams
        stored_ixs = []
        for j, ix in enumerate(ixs):
            # probability of paths ending with a non-blank
            pr_non_blank = 0
            # in case of non-empty beam
            if len(labels[ix]) > 0:
                # probability of paths with repeated last char at the end
                pr_non_blank = pr_non_blanks[ix] * mat[t, mapping[labels[ix][-1]]]

            # probability of paths ending with a blank
            pr_blank = pr_totals[ix] * mat[t, blank_idx]

            # fill in data for current beam
            new_labels.append(labels[ix])
            new_pr_non_blanks[j * max_C] = pr_non_blank
            new_pr_blanks[j * max_C] = pr_blank
            new_pr_totals[j * max_C] = pr_blank + pr_non_blank

            new_labeling = [labels[ix] + rev_mapping[c] for c in range(max_C - 1)]

            pr_non_blank = pr_totals[ix] * mat[t, last_chars_map]
            if len(labels[ix]) > 0:
                pr_non_blank[mapping[labels[ix][-1]]] = pr_blanks[ix] * mat[t, mapping[labels[ix][-1]]]

            new_labels += new_labeling
            new_pr_non_blanks[j * max_C + 1:(j + 1) * max_C] = pr_non_blank
            new_pr_totals[j * max_C + 1:(j + 1) * max_C] = pr_non_blank
            new_pr_blanks[j * max_C + 1:(j + 1) * max_C] = 0

        unique_labels, inverse = np.unique(new_labels, return_inverse=True, axis=0)
        label_count = unique_labels.shape[0]

        pr_blanks = np.zeros(label_count)
        pr_totals = np.zeros(label_count)
        pr_non_blanks = np.zeros(label_count)

        np.add.at(pr_blanks, inverse, new_pr_blanks[:len(new_labels)])
        np.add.at(pr_totals, inverse, new_pr_totals[:len(new_labels)])
        np.add.at(pr_non_blanks, inverse, new_pr_non_blanks[:len(new_labels)])

        labels = np.array([label for label in unique_labels])

    # sort by probability
    best_labeling = labels[np.argmax(pr_totals)]

    # map label string to char string
    # return np.array([mapping[c] for c in best_labeling], dtype=np.int64)
    return best_labeling

In [ ]:
def adjust_pad(pred, pad_rate):
  pred = tf.nn.softmax(pred)
  pred0, pred1 = pred[..., 0:1], pred[..., 1:]
  pred0 *= pad_rate
  pred = tf.concat([pred0, pred1], axis=-1)
  return pred

def ctc_decode(x):
  x = tf.concat([x, tf.zeros((1), dtype=x.dtype)], axis=0)
  diff = tf.not_equal(x[:-1], x[1:])
  adjacent_indices = tf.where(diff)[:, 0]
  x = tf.gather(x, adjacent_indices)
  mask = tf.math.logical_and(x > 0, x <= N_CHARS)
  x = tf.boolean_mask(x, mask, axis=0)
  return x

In [ ]:
def shift(logits):
  logits = tf.concat([logits[:,1:], logits[:,0:1]], axis=1)
  return logits

In [ ]:
def decode_phrase(pred, pad_rate=1):
  if pad_rate != 1:
    pred = adjust_pad(pred, pad_rate)
  x = tf.argmax(pred, axis=-1)
  x = ctc_decode(x)
  x = x.numpy()
  x = [IDX2CHAR[a] for a in x]
  return ''.join(x)
 

In [ ]:
from collections import defaultdict
from dataclasses import dataclass
from typing import Optional, List, Tuple

import numpy as np

# from ctc_decoder.language_model import LanguageModel


def log(x: float) -> float:
    with np.errstate(divide='ignore'):
        return np.log(x)


@dataclass
class BeamEntry:
    """Information about one single beam at specific time-step."""
    pr_total: float = log(0)  # blank and non-blank
    pr_non_blank: float = log(0)  # non-blank
    pr_blank: float = log(0)  # blank
    pr_text: float = log(1)  # LM score
    lm_applied: bool = False  # flag if LM was already applied to this beam
    labeling: tuple = ()  # beam-labeling


class BeamList:
    """Information about all beams at specific time-step."""

    def __init__(self) -> None:
        self.entries = defaultdict(BeamEntry)

    def normalize(self) -> None:
        """Length-normalise LM score."""
        for k in self.entries.keys():
            labeling_len = len(self.entries[k].labeling)
            self.entries[k].pr_text = (1.0 / (labeling_len if labeling_len else 1.0)) * self.entries[k].pr_text

    def sort_labelings(self) -> List[Tuple[int]]:
        """Return beam-labelings, sorted by probability."""
        beams = self.entries.values()
        sorted_beams = sorted(beams, reverse=True, key=lambda x: x.pr_total + x.pr_text)
        return [x.labeling for x in sorted_beams]


# def apply_lm(parent_beam: BeamEntry, child_beam: BeamEntry, chars: str, lm: LanguageModel) -> None:
#     """Calculate LM score of child beam by taking score from parent beam and bigram probability of last two chars."""
#     if not lm or child_beam.lm_applied:
#         return

#     # take bigram if beam length at least 2
#     if len(child_beam.labeling) > 1:
#         c = chars[child_beam.labeling[-2]]
#         d = chars[child_beam.labeling[-1]]
#         ngram_prob = lm.get_char_bigram(c, d)
#     # otherwise take unigram
#     else:
#         c = chars[child_beam.labeling[-1]]
#         ngram_prob = lm.get_char_unigram(c)

#     lm_factor = 0.01  # influence of language model
#     child_beam.pr_text = parent_beam.pr_text + lm_factor * log(ngram_prob)  # probability of char sequence
#     child_beam.lm_applied = True  # only apply LM once per beam entry


def beam_search(mat: np.ndarray, beam_width: int = 25) -> str:
    """Beam search decoder.

    See the paper of Hwang et al. and the paper of Graves et al.

    Args:
        mat: Output of neural network of shape TxC.
        chars: The set of characters the neural network can recognize, excluding the CTC-blank.
        beam_width: Number of beams kept per iteration.
        lm: Character level language model if specified.

    Returns:
        The decoded text.
    """

    blank_idx = len(chars)
    max_T, max_C = mat.shape

    # initialise beam state
    last = BeamList()
    labeling = ()
    last.entries[labeling] = BeamEntry()
    last.entries[labeling].pr_blank = log(1)
    last.entries[labeling].pr_total = log(1)

    # go over all time-steps
    for t in range(max_T):
        curr = BeamList()

        # get beam-labelings of best beams
        best_labelings = last.sort_labelings()[:beam_width]

        # go over best beams
        for labeling in best_labelings:

            # probability of paths ending with a non-blank
            pr_non_blank = log(0)
            # in case of non-empty beam
            if labeling:
                # probability of paths with repeated last char at the end
                pr_non_blank = last.entries[labeling].pr_non_blank + log(mat[t, labeling[-1]])

            # probability of paths ending with a blank
            pr_blank = last.entries[labeling].pr_total + log(mat[t, blank_idx])

            # fill in data for current beam
            curr.entries[labeling].labeling = labeling
            curr.entries[labeling].pr_non_blank = np.logaddexp(curr.entries[labeling].pr_non_blank, pr_non_blank)
            curr.entries[labeling].pr_blank = np.logaddexp(curr.entries[labeling].pr_blank, pr_blank)
            curr.entries[labeling].pr_total = np.logaddexp(curr.entries[labeling].pr_total,
                                                           np.logaddexp(pr_blank, pr_non_blank))
            curr.entries[labeling].pr_text = last.entries[labeling].pr_text
            curr.entries[labeling].lm_applied = True  # LM already applied at previous time-step for this beam-labeling

            # extend current beam-labeling
            for c in range(max_C - 1):
                # add new char to current beam-labeling
                new_labeling = labeling + (c,)

                # if new labeling contains duplicate char at the end, only consider paths ending with a blank
                if labeling and labeling[-1] == c:
                    pr_non_blank = last.entries[labeling].pr_blank + log(mat[t, c])
                else:
                    pr_non_blank = last.entries[labeling].pr_total + log(mat[t, c])

                # fill in data
                curr.entries[new_labeling].labeling = new_labeling
                curr.entries[new_labeling].pr_non_blank = np.logaddexp(curr.entries[new_labeling].pr_non_blank,
                                                                       pr_non_blank)
                curr.entries[new_labeling].pr_total = np.logaddexp(curr.entries[new_labeling].pr_total, pr_non_blank)

                # apply LM
                # apply_lm(curr.entries[labeling], curr.entries[new_labeling], chars, lm)

        # set new beam state
        last = curr

    # normalise LM scores according to beam-labeling-length
    last.normalize()

    # sort by probability
    best_labeling = last.sort_labelings()[0]  # get most probable labeling

    # map label string to char string
    res = ''.join([chars[label] for label in best_labeling if not chars[label].startswith('<')])
    return res

In [ ]:
decode_phrase(df.iloc[-4].logits, 0.6)

In [ ]:
beam_search(gezi.softmax(shift(adjust_pad(df.iloc[-4].logits, 0.6))), 3)

In [ ]:
# debug(-4)

In [ ]:
df_ = df.sample(frac=1).head(1000)

In [ ]:
phrase_preds = []
for logit in tqdm(df_.logits.values, total=len(df_)):
  phrase_preds.append(beam_search(gezi.softmax(shift(adjust_pad(logit, 0.6))), 3))
  # phrase_preds.append(decode_phrase(gezi.softmax(adjust_pad(logit, 0.6))))

In [ ]:
phrase_preds

In [ ]:
distances = []
for phrase_true, phrase_pred in tqdm(zip(df_.phrase_true, phrase_preds), total=len(df_)):
    distances.append(calc_score(phrase_true, phrase_pred))

In [ ]:
max(df_['phrase_len_true'].sum() - sum(df_['distance']), 0.) / df_['phrase_len_true'].sum()

In [ ]:
max(df_['phrase_len_true'].sum() - sum(distances), 0.) / df_['phrase_len_true'].sum()

In [ ]:
for i, (a, b) in enumerate(zip(df.phrase_pred, phrase_preds)):
  if a != b:
    print(i, a, '|', b)
    break

In [ ]:
df.iloc[405]

In [ ]:
decode_phrase(df.iloc[405].logits, 0.6)

In [ ]:
''.join([IDX2CHAR[a] for a in tf.argmax(adjust_pad(df.iloc[405].logits, 0.6), -1).numpy()])